## 1. Импорт библиотек и загрузка данных

In [ ]:
#загрузка необходимых библиотек
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
from sklearn import preprocessing
from sklearn.cluster import KMeans
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from sklearn.tree import DecisionTreeClassifier
from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
matplotlib.style.use('ggplot')
%matplotlib inline 
pd.set_option('display.float_format', lambda x: '%.1f' % x)

In [ ]:
dt = pd.read_csv('credit_train.csv',sep=(',') ,header=0)
dt.head(10)

,Loan ID,Customer ID,Loan Status,Current Loan Amount,Term,Credit Score,Annual Income,Years in current job,Home Ownership,Purpose,Monthly Debt,Years of Credit History,Months since last delinquent,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Maximum Open Credit,Bankruptcies,Tax Liens
0,14dd8831-6af5-400b-83ec-68e61888a048,981165ec-3274-42f5-a3b4-d104041a9ca9,Fully Paid,445412,Short Term,709.0,1167493.0,8 years,Home Mortgage,Home Improvements,5214.7,17.2,nan,6,1,228190,416746.0,1.0,0.0
1,4771cc26-131a-45db-b5aa-537ea4ba5342,2de017a3-2e01-49cb-a581-08169e83be29,Fully Paid,262328,Short Term,nan,nan,10+ years,Home Mortgage,Debt Consolidation,33296.0,21.1,8.0,35,0,229976,850784.0,0.0,0.0
2,4eed4e6a-aa2f-4c91-8651-ce984ee8fb26,5efb2b2b-bf11-4dfd-a572-3761a2694725,Fully Paid,99999999,Short Term,741.0,2231892.0,8 years,Own Home,Debt Consolidation,29200.5,14.9,29.0,18,1,297996,750090.0,0.0,0.0
3,77598f7b-32e7-4e3b-a6e5-06ba0d98fe8a,e777faab-98ae-45af-9a86-7ce5b33b1011,Fully Paid,347666,Long Term,721.0,806949.0,3 years,Own Home,Debt Consolidation,8741.9,12.0,nan,9,0,256329,386958.0,0.0,0.0
4,d4062e70-befa-4995-8643-a0de73938182,81536ad9-5ccf-4eb8-befb-47a4d608658e,Fully Paid,176220,Short Term,nan,nan,5 years,Rent,Debt Consolidation,20639.7,6.1,nan,15,0,253460,427174.0,0.0,0.0
5,89d8cb0c-e5c2-4f54-b056-48a645c543dd,4ffe99d3-7f2a-44db-afc1-40943f1f9750,Charged Off,206602,Short Term,7290.0,896857.0,10+ years,Home Mortgage,Debt Consolidation,16367.7,17.3,nan,6,0,215308,272448.0,0.0,0.0
6,273581de-85d8-4332-81a5-19b04ce68666,90a75dde-34d5-419c-90dc-1e58b04b3e35,Fully Paid,217646,Short Term,730.0,1184194.0,< 1 year,Home Mortgage,Debt Consolidation,10855.1,19.6,10.0,13,1,122170,272052.0,1.0,0.0
7,db0dc6e1-77ee-4826-acca-772f9039e1c7,018973c9-e316-4956-b363-67e134fb0931,Charged Off,648714,Long Term,nan,nan,< 1 year,Home Mortgage,Buy House,14806.1,8.2,8.0,15,0,193306,864204.0,0.0,0.0
8,8af915d9-9e91-44a0-b5a2-564a45c12089,af534dea-d27e-4fd6-9de8-efaa52a78ec0,Fully Paid,548746,Short Term,678.0,2559110.0,2 years,Rent,Debt Consolidation,18660.3,22.6,33.0,4,0,437171,555038.0,0.0,0.0
9,0b1c4e3d-bd97-45ce-9622-22732fcdc9a0,235c4a43-dadf-483d-aa44-9d6d77ae4583,Fully Paid,215952,Short Term,739.0,1454735.0,< 1 year,Rent,Debt Consolidation,39277.8,13.9,nan,20,0,669560,1021460.0,0.0,0.0


Переименуем столбцы для удобства работы с данными

In [ ]:
dttest = pd.read_csv('credit_test.csv',sep=(',') ,header=0)

Выведем информацию по нашим данным

In [ ]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 19 columns):
Loan ID                         100000 non-null object
Customer ID                     100000 non-null object
Loan Status                     100000 non-null object
Current Loan Amount             100000 non-null int64
Term                            100000 non-null object
Credit Score                    80846 non-null float64
Annual Income                   80846 non-null float64
Years in current job            95778 non-null object
Home Ownership                  100000 non-null object
Purpose                         100000 non-null object
Monthly Debt                    100000 non-null float64
Years of Credit History         100000 non-null float64
Months since last delinquent    46859 non-null float64
Number of Open Accounts         100000 non-null int64
Number of Credit Problems       100000 non-null int64
Current Credit Balance          100000 non-null int64
Maximum Open 

Выведем описание информации по столбцам

In [ ]:
dt.describe()

,Current Loan Amount,Credit Score,Annual Income,Monthly Debt,Years of Credit History,Months since last delinquent,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Maximum Open Credit,Bankruptcies,Tax Liens
count,100000.0,80846.0,80846.0,100000.0,100000.0,46859.0,100000.0,100000.0,100000.0,99998.0,99796.0,99990.0
mean,11760447.4,1076.5,1378276.6,18472.4,18.2,34.9,11.1,0.2,294637.4,760798.4,0.1,0.0
std,31783942.5,1475.4,1081360.2,12175.0,7.0,22.0,5.0,0.5,376170.9,8384503.5,0.4,0.3
min,10802.0,585.0,76627.0,0.0,3.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,179652.0,705.0,848844.0,10214.2,13.5,16.0,8.0,0.0,112670.0,273438.0,0.0,0.0
50%,312246.0,724.0,1174162.0,16220.3,16.9,32.0,10.0,0.0,209817.0,467874.0,0.0,0.0
75%,524942.0,741.0,1650663.0,24012.1,21.7,51.0,14.0,0.0,367958.8,782958.0,0.0,0.0
max,99999999.0,7510.0,165557393.0,435843.3,70.5,176.0,76.0,15.0,32878968.0,1539737892.0,7.0,15.0


Проверим данные на нулевые значения

In [ ]:
dt.isnull().sum()

Loan ID                             0
Customer ID                         0
Loan Status                         0
Current Loan Amount                 0
Term                                0
Credit Score                    19154
Annual Income                   19154
Years in current job             4222
Home Ownership                      0
Purpose                             0
Monthly Debt                        0
Years of Credit History             0
Months since last delinquent    53141
Number of Open Accounts             0
Number of Credit Problems           0
Current Credit Balance              0
Maximum Open Credit                 2
Bankruptcies                      204
Tax Liens                          10
dtype: int64

In [ ]:
dt.drop_duplicates(inplace=True)

In [ ]:
dt.describe()

,Current Loan Amount,Credit Score,Annual Income,Monthly Debt,Years of Credit History,Months since last delinquent,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Maximum Open Credit,Bankruptcies,Tax Liens
count,89785.0,70631.0,70631.0,89785.0,89785.0,41448.0,89785.0,89785.0,89785.0,89783.0,89595.0,89776.0
mean,13060894.7,1130.8,1375894.4,18396.9,18.2,35.0,11.1,0.2,294035.1,775656.5,0.1,0.0
std,33295559.6,1571.0,1104851.7,12145.3,7.0,22.0,5.0,0.5,372227.7,8803517.4,0.3,0.3
min,10802.0,585.0,76627.0,0.0,3.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,180268.0,708.0,847932.0,10157.4,13.5,16.0,8.0,0.0,112936.0,277068.0,0.0,0.0
50%,313874.0,729.0,1168975.0,16140.3,17.0,32.0,10.0,0.0,209722.0,472692.0,0.0,0.0
75%,532378.0,742.0,1648915.0,23918.9,21.7,51.0,14.0,0.0,367517.0,791450.0,0.0,0.0
max,99999999.0,7510.0,165557393.0,435843.3,70.5,176.0,76.0,15.0,32878968.0,1539737892.0,7.0,15.0


In [ ]:
dt.drop('Loan ID', axis=1, inplace=True)
dt.drop('Customer ID',axis=1, inplace=True)

In [ ]:
dttest.drop('Loan ID', axis=1, inplace=True)
dttest.drop('Customer ID',axis=1, inplace=True)

In [ ]:
from sklearn.preprocessing import LabelEncoder  

le = LabelEncoder()
dt['Loan Status']=le.fit_transform(dt['Loan Status'])

In [ ]:
from sklearn.preprocessing import LabelEncoder  

lek = LabelEncoder()
dt['Purpose']=lek.fit_transform(dt['Purpose'])
dt['Term']=lek.fit_transform(dt['Term'])
dt['Years in current job']=lek.fit_transform(dt['Years in current job'])
dt['Home Ownership']=lek.fit_transform(dt['Home Ownership'])

In [ ]:
from sklearn.preprocessing import LabelEncoder  

lem = LabelEncoder()
dttest['Purpose']=lem.fit_transform(dttest['Purpose'])
dttest['Term']=lem.fit_transform(dttest['Term'])
dttest['Years in current job']=lem.fit_transform(dttest['Years in current job'])
dttest['Home Ownership']=lem.fit_transform(dttest['Home Ownership'])

In [ ]:
dt['Current Loan Amount'].loc[dt['Current Loan Amount']>99999998]=-100

In [ ]:
dttest['Current Loan Amount'].loc[dttest['Current Loan Amount']>99999998]=-100

In [ ]:
dt.dropna(inplace=True)

In [ ]:
dt.describe()

,Loan Status,Current Loan Amount,Term,Credit Score,Annual Income,Years in current job,Home Ownership,Purpose,Monthly Debt,Years of Credit History,Months since last delinquent,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Maximum Open Credit,Bankruptcies,Tax Liens
count,32613.0,32613.0,32613.0,32613.0,32613.0,32613.0,32613.0,32613.0,32613.0,32613.0,32613.0,32613.0,32613.0,32613.0,32613.0,32613.0,32613.0
mean,0.8,262205.1,0.7,1120.3,1440675.3,4.4,1.9,3.8,19020.4,19.3,35.1,11.5,0.2,262919.5,678682.8,0.1,0.0
std,0.4,203102.7,0.4,1555.5,929507.6,3.2,0.9,2.3,12038.8,6.9,21.9,5.0,0.5,319256.1,5941252.0,0.4,0.3
min,0.0,-100.0,0.0,585.0,111245.0,0.0,0.0,0.0,0.0,3.8,0.0,1.0,0.0,0.0,0.0,0.0,0.0
25%,1.0,109054.0,0.0,705.0,913102.0,2.0,1.0,3.0,10904.1,14.6,16.0,8.0,0.0,100833.0,249018.0,0.0,0.0
50%,1.0,224796.0,1.0,725.0,1243778.0,3.0,1.0,3.0,16903.9,17.9,32.0,11.0,0.0,186314.0,423874.0,0.0,0.0
75%,1.0,391336.0,1.0,740.0,1722996.0,7.0,3.0,3.0,24498.2,22.6,51.0,14.0,0.0,326249.0,708928.0,0.0,0.0
max,1.0,789250.0,1.0,7510.0,30838995.0,11.0,3.0,15.0,229057.9,70.5,176.0,48.0,15.0,7350321.0,798255370.0,7.0,15.0


In [ ]:
X=dt.drop(['Loan Status'], axis=1)
y=dt['Loan Status']

In [ ]:
from sklearn.metrics import make_scorer
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix, accuracy_score, f1_score


def train_validate_model(model, X,y , grid_search_params):
       
    X_train, X_test, y_train, y_test = train_test_split(X,y,train_size = 0.8, random_state = 123)
    
    model_grid = GridSearchCV(model, 
                              grid_search_params,
                              scoring = make_scorer(f1_score,greater_is_better=True), # Установка метрики по которой будут сравниваться модели
                              cv=5, n_jobs=-1,verbose= 0)
    
    clf=model_grid.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    # Переменные для вывода и для сохранения
    model_type = str(type(model)).split('.')[-1].replace("'>",'') # Выделение названия модели из type(model)
    model_info = {
        'model': model_type,
        'f1_score' : model_grid.best_score_,
        'Best params': model_grid.best_params_
        }
    print('Тип модели: '+str(model_type))
    print('MAE: {}'.format(model_grid.best_score_))
    print('Best params: '+str(model_grid.best_params_))
    print('-----------------------------------\n')
    return model_grid.best_estimator_, model_info

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

dectree = DecisionTreeClassifier()
randfor = RandomForestClassifier()

dectree_params = {'min_samples_split':np.arange(2,20)}
randfor_params = {'min_samples_split':np.arange(2,20), 'max_depth':np.arange(2,20)}

models = [(dectree, dectree_params), (randfor, randfor_params)]

In [ ]:
model_info = []
estimators = []
for model in models:
    estimator, info = train_validate_model(model[0], X, y, model[1])
    model_info.append(info)
    estimators.append(estimator)

Тип модели: DecisionTreeClassifier
MAE: 0.824897965128
Best params: {'min_samples_split': 19}
-----------------------------------

Тип модели: RandomForestClassifier
MAE: 0.894545064439
Best params: {'min_samples_split': 19, 'max_depth': 8}
-----------------------------------



In [ ]:
dttest=dttest.fillna(-999)

In [ ]:
le.inverse_transform(estimators[1].predict(dttest))

array(['Fully Paid', 'Fully Paid', 'Fully Paid', ..., 'Fully Paid',
       'Fully Paid', 'Fully Paid'], dtype=object)

In [ ]:
dttest['Loan Status']=le.inverse_transform(estimators[1].predict(dttest))

In [ ]:
predict=dttest['Loan Status'].copy()

In [ ]:
dttest['Loan Status']=predict

In [ ]:
dttest[['Loan ID','Loan Status']].to_csv('Itog3.csv',index=False)

In [ ]:
dttest.to_csv('')